<a href="https://colab.research.google.com/github/anismustaffa/FreeCodeCamp-Pandas-Real-Life-Example/blob/master/Code_Yelp_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from collections import Counter
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
# Read the CSV files into DataFrames
business_file_path = '/content/gdrive/MyDrive/BA572/yelp_academic_dataset_business.csv'
review_file_path = '/content/gdrive/MyDrive/BA572/yelp_academic_dataset_review.csv'

business = pd.read_csv(business_file_path)
review = pd.read_csv(review_file_path)

# List the columns of the DataFrame
columns_list = business.columns.tolist()
print("Columns in the 'business' DataFrame:")
for col in columns_list:
    print(col)


<ipython-input-4-0c971c280691>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  review = pd.read_csv(review_file_path)


Columns in the 'business' DataFrame:
Unnamed: 0
business_id
name
address
city
state
postal_code
latitude
longitude
stars
review_count
is_open
attributes
categories
hours


In [5]:
# Split the categories in the 'categories' column and count the occurrences of each category
categories_counter = Counter([category.strip() for categories in business['categories'].dropna() for category in categories.split(',')])

# Get the top 10 most common categories
top_10_categories = categories_counter.most_common(10)

# Print the top 10 categories
print("Top 10 most common types of businesses:")
for category, count in top_10_categories:
    print(f"{category}: {count} businesses")

Top 10 most common types of businesses:
Restaurants: 52268 businesses
Food: 27781 businesses
Shopping: 24395 businesses
Home Services: 14356 businesses
Beauty & Spas: 14292 businesses
Nightlife: 12281 businesses
Health & Medical: 11890 businesses
Local Services: 11198 businesses
Bars: 11065 businesses
Automotive: 10773 businesses


In [6]:
# List the columns of the DataFrame
columns_list = review.columns.tolist()
print("Columns in the 'yelp_academic_dataset_review' DataFrame:")
for col in columns_list:
    print(col)

Columns in the 'yelp_academic_dataset_review' DataFrame:
Unnamed: 0
review_id
user_id
business_id
stars
useful
funny
cool
text
date


In [10]:
# Filter the 'business' DataFrame to include only rows with 'Shopping' in 'categories'
shopping_business = business[business['categories'].str.contains('Shopping', case=False, na=False)]

# Merge the filtered 'business' DataFrame with the 'review' DataFrame on 'business_id'
merged_data = pd.merge(shopping_business, review, on='business_id', how='inner')

# Check if 'stars_x' and 'stars_y' columns exist in the merged data
if 'stars_x' in merged_data.columns and 'stars_y' in merged_data.columns:
    # Create a new column 'merged_stars' by merging 'stars_x' and 'stars_y'
    merged_data['stars'] = merged_data.apply(lambda row: row['stars_x'] if pd.notna(row['stars_x']) else row['stars_y'], axis=1)

    # Drop 'stars_x' and 'stars_y' columns
    merged_data.drop(['stars_x', 'stars_y'], axis=1, inplace=True)
# Save the merged data into another file
merged_file_path = '/content/gdrive/MyDrive/BA572/shopping_business_reviews.csv'
merged_data.to_csv(merged_file_path, index=False)

print("Merged data saved successfully.")

Merged data saved successfully.


In [11]:
# Check the column names to ensure 'stars' column exists
print("Column names in the merged data:")
print(merged_data.columns)

# Separate the data into two DataFrames based on star ratings if 'stars' column exists
if 'stars' in merged_data.columns:
    low_rated_reviews = merged_data[(merged_data['stars'] == 1.0) | (merged_data['stars'] == 2.0)]
    high_rated_reviews = merged_data[(merged_data['stars'] == 4.0) | (merged_data['stars'] == 5.0)]

    # Save the separated DataFrames into new CSV files
    low_rated_file_path = '/content/gdrive/MyDrive/BA572/low_rated_reviews.csv'
    high_rated_file_path = '/content/gdrive/MyDrive/BA572/high_rated_reviews.csv'

    low_rated_reviews.to_csv(low_rated_file_path, index=False)
    high_rated_reviews.to_csv(high_rated_file_path, index=False)

    print("Separation and saving completed successfully.")
else:
    print("The 'stars' column does not exist in the merged data. Check the column names.")

Column names in the merged data:
Index(['Unnamed: 0_x', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'review_count', 'is_open',
       'attributes', 'categories', 'hours', 'Unnamed: 0_y', 'review_id',
       'user_id', 'useful', 'funny', 'cool', 'text', 'date', 'stars'],
      dtype='object')
Separation and saving completed successfully.
